In [1]:
# section 1 load all the necessary modules and packages
import glob
import time
import geopandas as gpd
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import Polygon
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt
import warnings
import sys
import os


In [2]:
# define the target river segment
target_segment = 71021602 # segment that passes Saskatoon


# netwrok topology without lakes

In [3]:
# load the data
cat = gpd.read_file('../data/shp/cat_'+str(target_segment)+'_merit.shp')
riv = gpd.read_file('../data/shp/riv_'+str(target_segment)+'_merit.shp')

# 
IDs = np.unique(np.append(riv.seg_id,cat.cat_id)) # merge the ID of segments, and lakes
data = {'ID': IDs}
NTOPO = pd.DataFrame(data=data)

# creaton of other field for NTOPO
NTOPO ['ID_next'] = -9999
NTOPO ['area']    = 1 # m2
NTOPO ['length']  = 1 # m
NTOPO ['slope']   = 0.001 # default value but can be read from riv


warnings.simplefilter('ignore') # silent the warning
for index, row in NTOPO.iterrows():
    
    if row.ID in np.array(riv['seg_id']): # the ID is a river segment
        
        # slice the riv and allocate the values
        riv_slice = riv[riv['seg_id']==row.ID]           
        # 
        NTOPO ['ID_next'].loc[index]  = riv_slice['to_seg_id'].iloc[0]
        NTOPO ['length'].loc[index]   = riv_slice['length'].iloc[0]
                       
    if row.ID in np.array(cat['cat_id']): # the cat ID            
        
        # slice the cat
        cat_slice = cat[cat['cat_id']==row.ID]
        # 
        NTOPO ['area'].loc[index]  = cat_slice['cat_area'].iloc[0]
warnings.simplefilter('default') # silent the warning
        
NTOPO.to_csv('../data/network_topology/Network_topology_merit_no_lake.csv')



In [4]:
# save the network topology
df = pd.read_csv('../data/network_topology/Network_topology_merit_no_lake.csv')

df = df.set_index('ID')

ds = df.to_xarray()

ds.attrs['Conventions'] = 'CF-1.6'
ds.attrs['License']     = 'The data were written by Shervan Gharari. They are under GPL.'
ds.attrs['history']     = 'Created ' + time.ctime(time.time())
ds.attrs['source']      = 'Written by test script of utilities (https://github.com/ShervanGharari/utility-codes'

var_info = {'ID':        {'long_name': 'ID',        'unit': '-'},
            'ID_next':   {'long_name': 'ID next',   'unit': '-'},
            'length':    {'long_name': 'length',    'unit': 'm'},
            'area':      {'long_name': 'area',      'unit': 'm**2'},
            'slope':     {'long_name': 'slope',     'unit': 'm m**-1'}}

var_encoding = {'ID':        {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                'ID_next':   {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                'length':    {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                'area':      {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                'slope':     {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9}}

for key in var_info.keys():
    for key1 in var_info[key].keys():
        print(var_info[key][key1])
        ds[key].attrs[key1] = var_info[key][key1]
        print(ds[key].attrs[key1])


if os.path.isfile('../data/network_topology/Network_topology_merit_no_lake.nc'):
    os.remove('../data/network_topology/Network_topology_merit_no_lake.nc')

ds = ds.rename_dims({'ID': 'n'})
ds.to_netcdf('../data/network_topology/Network_topology_merit_no_lake.nc',\
             encoding = var_encoding)

#
ds = xr.open_dataset('../data/network_topology/Network_topology_merit_no_lake.nc')

ds


ID
ID
-
-
ID next
ID next
-
-
length
length
m
m
area
area
m**2
m**2
slope
slope
m m**-1
m m**-1


<xarray.Dataset>
Dimensions:     (n: 2993)
Coordinates:
    ID          (n) float64 ...
Dimensions without coordinates: n
Data variables:
    Unnamed: 0  (n) int64 ...
    ID_next     (n) float64 ...
    area        (n) float64 ...
    length      (n) float64 ...
    slope       (n) float64 ...
Attributes:
    Conventions:  CF-1.6
    License:      The data were written by Shervan Gharari. They are under GPL.
    history:      Created Tue Aug 29 22:24:45 2023
    source:       Written by test script of utilities (https://github.com/She...